## Objective: testing the correctness of brain_region_signal_statistics_calculator using a synthetic data


In [1]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append("../")
from volume_statistics_calculalor.brain_region_signal_statistics_calculalor import BrainRegionSignalStatisticsCalculator


## Create synthetic data set

In [2]:
# signal volume and annotation volume
signal_file = './synthetic_data/signal.npz'
annotation_file = './synthetic_data/annotation.npz'

cols, rows = 4, 7
signal_volume = np.array([[i*j for i in range(cols)] for j in range(rows)])
print("signal_volume:")
print(signal_volume)

annotation_volume = np.array([[j for i in range(cols)] for j in range(rows)])
print("annotation_volume:")
print(annotation_volume)

np.savez_compressed(signal_file, signal_volume)
np.savez_compressed(annotation_file, annotation_volume)

# structure file
d = {'id': [1, 2, 3, 4, 5, 6], 
     'structure_id_path': ['/1/', 
                           '/1/2/', 
                           '/1/2/3/',
                           '/4/',
                           '/4/5/',
                           '/4/6/',
                          ]}

structures_df_path = './synthetic_data/structures.csv'
structures_df = pd.DataFrame(data=d)
structures_df.to_csv(structures_df_path, header=True)


signal_volume:
[[ 0  0  0  0]
 [ 0  1  2  3]
 [ 0  2  4  6]
 [ 0  3  6  9]
 [ 0  4  8 12]
 [ 0  5 10 15]
 [ 0  6 12 18]]
annotation_volume:
[[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]
 [5 5 5 5]
 [6 6 6 6]]


## Ground truth

```
Region structure
  +-- 1
  |   +-- 2 +-- 3
  +-- 4
  |   +-- 5
  |   +-- 6
```

* The number of leaf nodes without any children: 3
(id: 3, 5, 6)
* The number of unique labels in anaotation volume: 7
(id: 0, 1, 2, 3, 4, 5, 6)
* The number of valid labels in anaotation volume: 3
(id: 3, 5, 6)
* The number of invalid labels in anaotation volume: 4
(id: 0 (background), 1, 2, 4)

## Testing 

In [3]:
calculator = BrainRegionSignalStatisticsCalculator('synthetic_data/annotation.npz', 
                                                   'synthetic_data/signal.npz', 
                                                   'synthetic_data/structures.csv', 
                                                   'synthetic_data/statistics_ground_truth.csv')
brain_statistics = calculator.calculate()

# of leaf nodes without any children: 3
# of unique labels in anaotation volume: 7
# of valid labels in anaotation volume: 3
# of invalid labels in anaotation volume: 4


In [4]:
brain_statistics

{1: {'mean': 4.5, 'min': 0, 'max': 9, 'sum': 18, 'count': 4},
 2: {'mean': 4.5, 'min': 0, 'max': 9, 'sum': 18, 'count': 4},
 3: {'mean': 4.5, 'min': 0, 'max': 9, 'sum': 18, 'count': 4},
 4: {'mean': 8.25, 'min': 0, 'max': 18, 'sum': 66, 'count': 8},
 5: {'mean': 7.5, 'min': 0, 'max': 15, 'sum': 30, 'count': 4},
 6: {'mean': 9.0, 'min': 0, 'max': 18, 'sum': 36, 'count': 4}}